## Import libraries

In [1]:
import numpy as np
import tensorflow as tf
import time

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### Helper functions

In [2]:
def dh_matrix(a, alpha, d, theta):
    """Calculate the DH matrix for given parameters."""
    theta = np.deg2rad(theta)
    alpha = np.deg2rad(alpha)
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])

def forward_kinematics(thetas, dh_params):
    """Calculate forward kinematics for given joint angles and DH parameters."""
    T = np.eye(4)
    positions = [np.array([0, 0, 0])]
    
    for i in range(len(dh_params)):
        a, alpha, d, theta_offset = dh_params[i]
        theta = thetas[i] + theta_offset
        T_i = dh_matrix(a, alpha, d, theta)
        T = np.dot(T, T_i)
        positions.append(T[:3, 3])
    
    return np.array(positions), T

def dh_matrix_tf(a, alpha, d, theta):
    """Calculate the DH matrix for given parameters using TensorFlow."""
    # Convert degrees to radians
    theta = theta * np.pi / 180.0
    alpha = alpha * np.pi / 180.0
    
    cos_theta, sin_theta = tf.cos(theta), tf.sin(theta)
    cos_alpha, sin_alpha = tf.cos(alpha), tf.sin(alpha)
    
    # Ensure all inputs have the same shape for broadcasting
    zeros = tf.zeros_like(theta)
    ones = tf.ones_like(theta)
    
    return tf.stack([
        tf.stack([cos_theta, -sin_theta*cos_alpha, sin_theta*sin_alpha, a*cos_theta], axis=-1),
        tf.stack([sin_theta, cos_theta*cos_alpha, -cos_theta*sin_alpha, a*sin_theta], axis=-1),
        tf.stack([zeros, sin_alpha, cos_alpha, d*ones], axis=-1),
        tf.stack([zeros, zeros, zeros, ones], axis=-1)
    ], axis=-2)

def forward_kinematics_TF(thetas_batch, dh_params):
    """
    Calculate forward kinematics for a batch of joint angles using TensorFlow.
    
    Args:
    thetas_batch: TensorFlow tensor of shape (batch_size, num_joints)
    dh_params: NumPy array of shape (num_joints, 4) containing DH parameters [a, alpha, d, theta_offset]
    
    Returns:
    positions: TensorFlow tensor of shape (batch_size, num_joints + 1, 3)
    T: TensorFlow tensor of shape (batch_size, 4, 4) - final transformation matrices
    """
    batch_size = tf.shape(thetas_batch)[0]
    num_joints = tf.shape(thetas_batch)[1]
    
    # Convert DH parameters to TensorFlow tensors and broadcast to batch size
    dh_params_tf = tf.constant(dh_params, dtype=tf.float32)
    dh_params_tf = tf.broadcast_to(dh_params_tf[None, :, :], [batch_size, num_joints, 4])
    
    # Initialize transformation matrix
    T = tf.eye(4, batch_shape=[batch_size])
    
    # Initialize positions tensor
    positions = tf.zeros((batch_size, num_joints + 1, 3))
    
    for i in range(num_joints):
        a, alpha, d, theta_offset = tf.unstack(dh_params_tf[:, i], axis=-1)
        theta = thetas_batch[:, i] + theta_offset
        T_i = dh_matrix_tf(a, alpha, d, theta)
        T = tf.matmul(T, T_i)
        positions = tf.tensor_scatter_nd_update(
            positions, 
            tf.stack([tf.range(batch_size), tf.fill([batch_size], i+1)], axis=1),
            T[:, :3, 3]
        )
    
    return positions, T

### Initialize DH parameters

In [3]:
dh_params = np.array([
    [0, 90, 0.0, 0],   # Joint 1
    [50, 0, 0, 90],    # Joint 2
    [50, 0, 0, -90],   # Joint 3
    [0, 90, 10, -90],  # Joint 4
    [0, -90, 10, 0],   # Joint 5
    [0, 0, 10, 0]      # Joint 6
])

def run_and_time_numpy(num_iterations):
    thetas = np.random.uniform(-np.pi, np.pi, (num_iterations, 6))
    start_time = time.time()
    for i in range(num_iterations):
        positions_np, T_np = forward_kinematics(thetas[i], dh_params)
    end_time = time.time()
    return end_time - start_time

def run_and_time_tensorflow(batch_size):
    thetas_batch = tf.random.uniform((batch_size, 6), minval=-np.pi, maxval=np.pi)
    start_time = time.time()
    positions_tf, T_tf = forward_kinematics_TF(thetas_batch, dh_params)
    end_time = time.time()
    return end_time - start_time

### Test and time the NumPy version

In [4]:
print("NumPy version test (single iteration):")
thetas = np.zeros(6)  # Example joint angles
positions_np, T_np = forward_kinematics(thetas, dh_params)

print("Joint positions:")
for i, pos in enumerate(positions_np):
    print(f"Joint {i}: {pos}")
print("\nFinal transformation matrix:")
print(T_np)

NumPy version test (single iteration):
Joint positions:
Joint 0: [0. 0. 0.]
Joint 1: [0. 0. 0.]
Joint 2: [3.061617e-15 3.061617e-15 5.000000e+01]
Joint 3: [5.000000e+01 3.061617e-15 5.000000e+01]
Joint 4: [ 50. -10.  50.]
Joint 5: [ 40. -10.  50.]
Joint 6: [ 40. -20.  50.]

Final transformation matrix:
[[ 6.123234e-17  1.000000e+00  0.000000e+00  4.000000e+01]
 [-6.123234e-17  0.000000e+00 -1.000000e+00 -2.000000e+01]
 [-1.000000e+00  6.123234e-17  6.123234e-17  5.000000e+01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]


### Test and time the TensorFlow batch version

In [5]:
print("\nTensorFlow batch version test (single batch):")
batch_size = 2048
thetas_batch = tf.random.uniform((batch_size, 6), minval=-np.pi, maxval=np.pi)
positions_tf, T_tf = forward_kinematics_TF(thetas_batch, dh_params)

print("Joint positions (first item in batch):")
for i in range(7):
    print(f"Joint {i}: {positions_tf[0, i].numpy()}")
print("\nFinal transformation matrix (first item in batch):")
print(T_tf[0].numpy())


TensorFlow batch version test (single batch):
Joint positions (first item in batch):
Joint 0: [0. 0. 0.]
Joint 1: [0. 0. 0.]
Joint 2: [-1.7374598e+00  2.1479061e-02  4.9969799e+01]
Joint 3: [48.17449    -0.59561217 52.870895  ]
Joint 4: [ 48.050865 -10.594849  52.870895]
Joint 5: [ 38.06524  -10.47139   52.349346]
Joint 6: [ 37.93683  -20.470146  52.441025]

Final transformation matrix (first item in batch):
[[ 3.2372497e-02  9.9939334e-01 -1.2840903e-02  3.7936829e+01]
 [-9.5796892e-03 -1.2536790e-02 -9.9987555e-01 -2.0470146e+01]
 [-9.9942994e-01  3.2491483e-02  9.1680316e-03  5.2441025e+01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


### Timing comparisons

In [6]:
print("\nTiming Comparisons:")

num_iterations = 2**16
numpy_time = run_and_time_numpy(num_iterations)
print(f"NumPy version (sequential, {num_iterations} iterations): {numpy_time:.4f} seconds")

tensorflow_time = run_and_time_tensorflow(num_iterations)
print(f"TensorFlow version (batched, batch size {num_iterations}): {tensorflow_time:.4f} seconds")

speedup = numpy_time / tensorflow_time
print(f"\nSpeedup factor: {speedup:.2f}x")

print("\nNote: Run this in a Jupyter notebook for more accurate timing results.")


Timing Comparisons:
NumPy version (sequential, 65536 iterations): 6.1924 seconds
TensorFlow version (batched, batch size 65536): 0.0214 seconds

Speedup factor: 288.79x

Note: Run this in a Jupyter notebook for more accurate timing results.
